In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from datasets import Dataset

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# from transformers import DataCollatorForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments

In [2]:
df = pd.read_csv("data/data_.csv")
df

,output,input
0,loneliness\nhis childhood\nwarms him up\nlonel...,generate a poem by the following author: johnt...
1,Sit in a chair\nalone\nPeople I thought were\n...,generate a poem by the following author: Deana...
2,Tell me why it has to be this way\nwhy I must ...,generate a poem by the following author: Deana...
3,"A debtor to mercy alone, of covenant mercy I s...",generate a poem by the following author: Augus...
4,A year has been passed since I left home\nAgai...,generate a poem by the following author: RaviS...
...,...,...
14330,"We may roam through this world, like a child a...",generate a poem by the following author: Thoma...
14331,When you were born in this world\nEveryone lau...,generate a poem by the following author: Kabir...
14332,"When I cast a look around the world,\nI marvel...",generate a poem by the following author: MacAr...
14333,"WORLD, take good notice, silver stars fading,\...",generate a poem by the following author: WaltW...


In [3]:
model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\HamzaJadoon\anaconda3\envs\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
data_dict = df.to_dict(orient='list')
dataset = Dataset.from_dict(data_dict)
dataset

Dataset({
    features: ['output', 'input'],
    num_rows: 14335
})

In [5]:
def prepare_data(data):

    # print("\n\n\nKeys:", list(data.keys()))
    # for key, value in data.items():
    #     print(".................................................")
    #     print("here is key ---->>>",key)
    #     print("here is value ---->>>",value)
        
    model_inputs = tokenizer(data["input"], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(data["output"], max_length=512, truncation=True, padding="max_length")
    model_inputs["label"] = labels["input_ids"]
    
    # print("\n\n\nKeys:", list(model_inputs.keys()))
    # print("labels:", list(labels.keys()))
    # print(model_inputs["input_ids"])
    # print(model_inputs["attention_mask"])
    # print(labels["attention_mask"])

    return model_inputs

# tokenized_dataset has tokenized inputs, their attention masks and tokenized labels
tokenized_dataset = dataset.map(prepare_data)

# for row in dataset["content"]:
#     # prepare_data({"content": row})
#     prepare_data(row)
#     break 

Map:   0%|          | 0/14335 [00:00<?, ? examples/s]

In [6]:
tokenized_dataset

Dataset({
    features: ['output', 'input', 'input_ids', 'attention_mask', 'label'],
    num_rows: 14335
})

In [7]:
# print(tokenized_dataset['input'][0])
# print(tokenized_dataset['output'][0])

# print('\n',tokenized_dataset['input_ids'][0])
# print(tokenized_dataset['labels'][0])

In [8]:
# Remove unnecessary columns
columns_to_remove = ['output', 'input']
tokenized_dataset = tokenized_dataset.remove_columns(columns_to_remove)

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Set the format of your dataset
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Check a few examples
# print(train_dataset[0])
# print(tokenizer.decode(train_dataset[0]['input_ids']))
# print(tokenizer.decode(train_dataset[0]['label_ids']))

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [9]:
print(train_dataset[12000].keys())

dict_keys(['input_ids', 'attention_mask', 'label'])


In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

c:\Users\HamzaJadoon\anaconda3\envs\myenv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)